In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site

os.sys.path.insert(0, '/home/schirrmr/code/hyperoptim//')
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

In [ ]:
from hyperoptim.results import (load_data_frame,
                                remove_columns_with_same_value, mean_identical_exps, pairwise_compare_frame,
                                round_numeric_columns)
import pandas as pd

def load_df(folder):
    df = load_data_frame(folder)
    df = df.drop(['save_folder', 'seed', 'only_return_exp'], axis=1)
    df = df[df.finished == 1]
    df = df[df.debug == 0]
    if len(df) > 0:
        for key in ['train_acc', 'train_data_acc','test_acc', 'test_data_acc', 'test_clf_acc', 'train_clf_acc']:
            if key in df.columns:
                df.loc[:,key] = np.round(df.loc[:,key] * 100,1)
        df = remove_columns_with_same_value(df) 
        df.runtime = pd.to_timedelta(np.round(df.runtime), unit='s')
        if 'clf_loss' in df.columns:
            df.clf_loss = df.clf_loss.replace([None], 'none')

    print(len(df))
    return df

In [ ]:
result_cols = ['g_loss', 'ot_in_loss_other_1',
               'runtime', 'test_data_acc', 'test_acc', 'train_data_acc', 'ot_out_loss',
       'ot_in_loss_1', 'train_acc', 'g_grad', 'ot_in_loss_other_0',
       'ot_loss_in_1', 'g_grad_norm', 'ot_in_loss_0', 'ot_loss_in_0',
       'ot_out_loss_other', 'clf_loss_0',  'clf_loss_1','subspace_loss_0',
               'subspace_loss_1', 'test_clf_acc',
              'train_clf_acc']

interesting_result_cols = [
               'runtime', 'test_data_acc', 'test_acc', 'test_clf_acc',  'ot_out_loss',
        'ot_loss_in_0', 'ot_loss_in_1',
       'subspace_loss_0',
               'subspace_loss_1', 
              'train_data_acc', 'train_acc','train_clf_acc', ]

In [ ]:
df = load_df('/data/schirrmr/schirrmr/reversible/experiments/new-deep-invertible//')
df = mean_identical_exps(df, result_cols=result_cols).drop('n_exp', axis=1)

meaned_df = mean_identical_exps(df.drop('subject_id', axis=1), result_cols=result_cols)
for c in result_cols:
    if c in meaned_df.columns and c not in interesting_result_cols:
        meaned_df = meaned_df.drop(c, axis=1)
meaned_df = round_numeric_columns(meaned_df, 1)
meaned_df

In [ ]:
df = load_df('/data/schirrmr/schirrmr/reversible/experiments/for-poster-2///')
meaned_df = mean_identical_exps(df.drop('subject_id', axis=1), result_cols=result_cols).loc[
    :, ['n_exp', 'max_epochs',] + 
                                              interesting_result_cols]
meaned_df

In [ ]:
df = load_df('/data/schirrmr/schirrmr/reversible/experiments/classifier/')
df.clf_loss = df.clf_loss.replace([None], 'none')
df = df[df.ot_on_class_dims == df.ot_on_class_dims] # remove nan rows
df = remove_columns_with_same_value(df)
df = mean_identical_exps(df, result_cols=result_cols).drop('n_exp', axis=1)

df.drop(['g_grad','g_grad_norm'], axis=1).loc[:,['subject_id', 'ot_on_class_dims', 'clf_loss', 'final_hz', 'n_sensors'] + 
                                              interesting_result_cols]
meaned_df = mean_identical_exps(df.drop('subject_id', axis=1), result_cols=result_cols).loc[
    :, ['n_exp', 'ot_on_class_dims', 'clf_loss', 'final_hz', 'n_sensors'] + 
                                              interesting_result_cols]
#meaned_df.sort_values(by=['final_hz', 'n_sensors', 'clf_loss', 'ot_on_class_dims'])
meaned_df[(meaned_df.final_hz == 256) &
         (meaned_df.n_sensors == 22)]

In [ ]:
df.subject_idct_id